In [1]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain.chat_models import ChatOpenAI
load_dotenv()

True

In [6]:
from dotenv import load_dotenv


In [8]:
KEY=os.getenv("OPEN_API_KEY")

In [9]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

/Users/vaibhavagarwal/Desktop/yash/mcqgen/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [23]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [21]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [18]:
quiz=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [19]:
quiz_chain=LLMChain(llm=llm,prompt=quiz,output_key="quiz",verbose=True)

/Users/vaibhavagarwal/Desktop/yash/mcqgen/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [20]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [37]:
quiz_eval=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE)


In [38]:
review_chain=LLMChain(llm=llm,prompt=quiz_eval,output_key="review",verbose=True)

In [39]:
eval_chain=SequentialChain(chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],
    verbose=True
)

In [40]:
file_path=r"/Users/vaibhavagarwal/Desktop/yash/mcqgen/data.txt"

In [41]:
file_path

'/Users/vaibhavagarwal/Desktop/yash/mcqgen/data.txt'

In [42]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [30]:
print(TEXT)

OpenAI is an American artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work."[4] As one of the leading organizations of the AI boom,[5][6][7] it has developed several large language models, advanced image generation models, and previously, released open-source models.[8][9] Its release of ChatGPT has been credited with starting the AI boom.[10]

The organization consists of the non-profit OpenAI, Inc.[11] registered in Delaware and its for-profit subsidiary OpenAI Global, LLC.[12] It was founded by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy, Durk Kingma, Jessica Livingston, John Schulman, Pamela Vagata, and Wojciech Zaremba, with Sam Altman and Elon Musk serving as the initial Board of Directors members.[13]

In [43]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [44]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [45]:
with get_openai_callback() as cb:
    response=eval_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:OpenAI is an American artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work."[4] As one of the leading organizations of the AI boom,[5][6][7] it has developed several large language models, advanced image generation models, and previously, released open-source models.[8][9] Its release of ChatGPT has been credited with starting the AI boom.[10]

The organization consists of the non-profit OpenAI, Inc.[11] registered in Delaware and its for-profit subsidiary OpenAI Global, LLC.[12] It was founded by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy, Durk Kingma, Jessica Livingston, John Schulman, Pamela Vagata

In [35]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1531
Prompt Tokens:1102
Completion Tokens:429
Total Cost:0.002511


In [46]:
response

{'text': 'OpenAI is an American artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work."[4] As one of the leading organizations of the AI boom,[5][6][7] it has developed several large language models, advanced image generation models, and previously, released open-source models.[8][9] Its release of ChatGPT has been credited with starting the AI boom.[10]\n\nThe organization consists of the non-profit OpenAI, Inc.[11] registered in Delaware and its for-profit subsidiary OpenAI Global, LLC.[12] It was founded by Ilya Sutskever, Greg Brockman, Trevor Blackwell, Vicki Cheung, Andrej Karpathy, Durk Kingma, Jessica Livingston, John Schulman, Pamela Vagata, and Wojciech Zaremba, with Sam Altman and Elon Musk serving as the initial Board of Directors 

In [48]:
exam_quiz=response.get("quiz");
exam_quiz=json.loads(exam_quiz)
exam_quiz

{'1': {'mcq': 'Who founded OpenAI?',
  'options': {'a': 'Bill Gates',
   'b': 'Elon Musk',
   'c': 'Mark Zuckerberg',
   'd': 'Jeff Bezos'},
  'correct': 'b'},
 '2': {'mcq': 'What is the goal of OpenAI?',
  'options': {'a': 'To develop artificial general stupidity',
   'b': 'To develop safe and beneficial artificial general intelligence',
   'c': 'To outperform humans in all tasks',
   'd': 'To create chaos in the AI world'},
  'correct': 'b'},
 '3': {'mcq': 'Which company provided a $1 billion investment to OpenAI in 2019?',
  'options': {'a': 'Google', 'b': 'Apple', 'c': 'Microsoft', 'd': 'Amazon'},
  'correct': 'c'},
 '4': {'mcq': 'Who was removed as CEO of OpenAI in November 2023?',
  'options': {'a': 'Elon Musk',
   'b': 'Greg Brockman',
   'c': 'Sam Altman',
   'd': 'Ilya Sutskever'},
  'correct': 'c'},
 '5': {'mcq': 'Who was announced as the new chairman of OpenAI after the board changes in November 2023?',
  'options': {'a': 'Elon Musk',
   'b': 'Bret Taylor',
   'c': 'Bill Gat

In [51]:
# table=[]
# for key,value in exam_quiz.items():
#     mcq=value["mcq"]
#     options="      "  .join(
#         [
#             f"{option}:{option_value}"
#             for option ,option_value in value["options"].items()
#         ]
#     )
#     correct=value["correct"]
#     table.append({"MCQ":mcq,"CHOICES":options,"CORRECT":correct})
table = []

for key, value in exam_quiz.items():
    mcq = value["mcq"]
    options = ""
    for opt, val in value["options"].items():
        options += opt + ": " + str(val) + "      "
    options = options.rstrip("      ")  
    correct = value["correct"]
    table.append({"MCQ": mcq, "CHOICES": options, "CORRECT": correct})



In [52]:
table

[{'MCQ': 'Who founded OpenAI?',
  'CHOICES': 'a: Bill Gates      b: Elon Musk      c: Mark Zuckerberg      d: Jeff Bezos',
  'CORRECT': 'b'},
 {'MCQ': 'What is the goal of OpenAI?',
  'CHOICES': 'a: To develop artificial general stupidity      b: To develop safe and beneficial artificial general intelligence      c: To outperform humans in all tasks      d: To create chaos in the AI world',
  'CORRECT': 'b'},
 {'MCQ': 'Which company provided a $1 billion investment to OpenAI in 2019?',
  'CHOICES': 'a: Google      b: Apple      c: Microsoft      d: Amazon',
  'CORRECT': 'c'},
 {'MCQ': 'Who was removed as CEO of OpenAI in November 2023?',
  'CHOICES': 'a: Elon Musk      b: Greg Brockman      c: Sam Altman      d: Ilya Sutskever',
  'CORRECT': 'c'},
 {'MCQ': 'Who was announced as the new chairman of OpenAI after the board changes in November 2023?',
  'CHOICES': 'a: Elon Musk      b: Bret Taylor      c: Bill Gates      d: Jeff Bezos',
  'CORRECT': 'b'}]

In [54]:
exam_quiz=pd.DataFrame(table)
exam_quiz

,MCQ,CHOICES,CORRECT
0,Who founded OpenAI?,a: Bill Gates b: Elon Musk c: Mark Z...,b
1,What is the goal of OpenAI?,a: To develop artificial general stupidity ...,b
2,Which company provided a $1 billion investment...,a: Google b: Apple c: Microsoft ...,c
3,Who was removed as CEO of OpenAI in November 2...,a: Elon Musk b: Greg Brockman c: Sam...,c
4,Who was announced as the new chairman of OpenA...,a: Elon Musk b: Bret Taylor c: Bill ...,b


In [ ]:
exam_quiz.to_csv("quizesss.csv",index=False)